In [1]:
import polars as pl

In [3]:
df = pl.read_json('data_raw/papezstvo_raw.json')

In [9]:
mista_narozeni = df.select(pl.col('narozeni_misto')).unique().drop_nulls().to_series().to_list()
mista_narozeni

['Q191115',
 'Q278199',
 'Q40416',
 'Q159605',
 'Q124067',
 'Q91447',
 'Q243168',
 'Q6259',
 'Q199160',
 'Q13373',
 'Q2802461',
 'Q242675',
 'Q2348968',
 'Q104074',
 'Q13375',
 'Q2656',
 'Q272897',
 'Q13413',
 'Q155019',
 'Q13415123',
 'Q13376',
 'Q50157',
 'Q501773',
 'Q528042',
 'Q131301',
 'Q5475',
 'Q2044',
 'Q40219',
 'Q2751',
 'Q83113',
 'Q643907',
 'Q13670',
 'Q1231483',
 'Q53952',
 'Q41',
 'Q270613',
 'Q242720',
 'Q66764237',
 'Q42975',
 'Q55015',
 'Q124925',
 'Q16538015',
 'Q164092',
 'Q6226',
 'Q243458',
 'Q18493402',
 'Q1462',
 'Q40588',
 'Q51829',
 'Q1449',
 'Q91341',
 'Q589578',
 'Q1330965',
 'Q17283',
 'Q605051',
 'Q200441',
 'Q242000',
 'Q779478',
 'Q54513',
 'Q2968938',
 'Q2969360',
 'Q3632863',
 'Q873126',
 'Q5489',
 'Q693973',
 'Q641',
 'Q13437',
 'Q98308672',
 'Q47611',
 'Q1891',
 'Q3947656',
 'Q1273',
 'Q980',
 'Q73649',
 'Q242668',
 'Q101616',
 'Q59285',
 'Q1524',
 'Q490',
 'Q212856',
 'Q597',
 'Q2277',
 'Q56071',
 'Q831022',
 'Q463459',
 'Q20413',
 'Q270492',
 'Q5

In [11]:
import requests

In [15]:
def get_wikidata_info(wikidata_id):
    """
    Get coordinates and country information for a Wikidata ID.
    
    Args:
        wikidata_id (str): Wikidata ID (e.g., 'Q237')
        
    Returns:
        dict: Dictionary containing coordinates and country information
    """
    # Remove the 'Q' prefix if present in function calls
    if not wikidata_id.startswith('Q'):
        wikidata_id = f'Q{wikidata_id}'
        
    # Method 1: Direct API call
    url = f"https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "ids": wikidata_id,
        "format": "json",
        "props": "claims|labels"
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if "entities" not in data or wikidata_id not in data["entities"]:
        return {"error": "Entity not found"}
    
    entity = data["entities"][wikidata_id]
    claims = entity.get("claims", {})
    
    result = {
        "wikidata_id": wikidata_id,
        "name": entity.get("labels", {}).get("en", {}).get("value", "Unknown")
    }
    
    # Get coordinates (P625)
    if "P625" in claims:
        coord_claim = claims["P625"][0]
        if "datavalue" in coord_claim["mainsnak"]:
            coord_value = coord_claim["mainsnak"]["datavalue"]["value"]
            result["coordinates"] = {
                "latitude": coord_value["latitude"],
                "longitude": coord_value["longitude"]
            }
    
    # Get country (P17)
    if "P17" in claims:
        country_claim = claims["P17"][0]
        if "datavalue" in country_claim["mainsnak"]:
            country_id = country_claim["mainsnak"]["datavalue"]["value"]["id"]
            result["country_id"] = country_id
            
            # Get country name with a separate request
            country_url = f"https://www.wikidata.org/w/api.php"
            country_params = {
                "action": "wbgetentities",
                "ids": country_id,
                "format": "json",
                "props": "labels"
            }
            country_response = requests.get(country_url, params=country_params)
            country_data = country_response.json()
            
            if "entities" in country_data and country_id in country_data["entities"]:
                country_name = country_data["entities"][country_id].get("labels", {}).get("en", {}).get("value", "Unknown")
                result["country_name"] = country_name
    
    return result

In [17]:
get_wikidata_info('Q3995920')

{'wikidata_id': 'Q3995920',
 'name': 'Tossignano',
 'coordinates': {'latitude': 44.272222222222, 'longitude': 11.603611111111},
 'country_id': 'Q38',
 'country_name': 'Italy'}

In [19]:
mista = []
for m in mista_narozeni:
    mista.append(get_wikidata_info(m))

In [20]:
import json

In [22]:
import os

In [26]:
with open(os.path.join('data_raw','mista_narozeni.json'), 'w+', encoding='utf-8') as export:
    export.write(json.dumps(mista))